In [1]:
from urllib.request import urlopen
from lxml import etree
import re
import xlwt
import xlrd
from xlutils.copy import copy
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import requests
import json
import time
import sys
import traceback
from multiprocessing import Pool

In [14]:
# 使用了各种方法，去请求网页数据，都会被网站屏蔽。
# 所以这里采用自动化测试点击下一页来获取每页数据
browser = webdriver.Chrome()

In [15]:
#沪市、深市、中小板、创业板
# browser.get("http://data.10jqka.com.cn/financial/yjgg/###")
browser.get("http://quote.eastmoney.com/center/gridlist.html#hs_a_board")
#上证50、沪深300、中证500
# browser.get('http://data.eastmoney.com/other/index/hs300.html')

In [16]:
# wait = WebDriverWait(browser, 10)

In [17]:
# workbook = copy(xlrd.open_workbook("data/上市公司.xls"))
workbook = xlwt.Workbook()

In [18]:
#添加表头
def addHeaders(board_name,heads):
    # StockCode：股票代码
    # StockName：股票简称
    # Price：当前股价
    '-------每日浮动数据-------'
    # PriceLimit：涨跌幅
    # QuantityRelativeR：量比
    # TurnoverRate：换手率
    '-------相对固定数据-------'
    # Trade：行业
    # PB：市净率
    # PE_S：静态市盈率
    # PE_D：动态市盈率
    # EarningsPerShare：每股收益
    # NetProfitDes：净利润描述（包括净利润和增长比率）
    # ROE：净资产收益率
    # GrossProfitRate：毛利率
    # NetAssetValuePerShare：每股净资产
    # CapitalStock：股本
    # ScaleShareType：类型（大盘股、小盘股）
    # FinanceAnalize：财务分析
    '-------备用字段-------'
    # Proceeds：营业收入
    # ProceedsYOY：营业收入-同比增长
    # ProceedsQOQ：营业收入-环比增长
    # NetProfit：净利润
    # NetProfitYOY：净利润-同比增长
    # NetProfitQOQ：净利润-环比增长
    # MoneyFlowPerShare：每股现金流量
    # GrossProfitRate：销售毛利率

    sheet = workbook.add_sheet(board_name) 
    for h in range(len(heads)):
        sheet.write(0, h, heads[h]) 
    return sheet

# 获取股票数据
# def getStockDataByType(stock_type):
#     boards = browser.find_elements_by_class_name("J-board-item")
    
#     for i,board in enumerate(boards):
#         board_name =board.text
#         if board_name == stock_type:
#             sheet = addHeaders(board_name)
#             board.click()
#             time.sleep(3)
#             print("正在获取："+board_name+"...")
#             total_page = int(browser.find_element_by_class_name("page_info").text.split("/")[1])
#             for page in range(1,total_page+1):
#                 comapnies = browser.find_elements_by_class_name("J_showCanvas")
#                 for j,company in enumerate(comapnies):
#                     printProgress("第"+str(page)+"页-"+"第"+str(j)+"条...")
#                     code = browser.find_element_by_xpath('//*[@id="J-ajax-main"]/table/tbody/tr['+str(j+1)+']/td[2]/a')
#                     row = (page-1)*50+1+j
#                     sheet.write(row,0,code.text)
#                     sheet.write(row,1,company.text)
#                     getBaseData(code.text,row,2,sheet)
                
#                 page_btns = browser.find_elements_by_class_name("changePage")
#                 subscript = 0
#                 if page != total_page+1:
#                     for page_btn in page_btns:
#                         if page_btn.text == "下一页":
#                             subscript = page_btns.index(page_btn)
#                     page_btns[subscript].click()
#                     time.sleep(3)
                    
#     browser.close()
#     browser.quit()

def getStockDataByType(stock_type):
    heads = ['StockCode','StockName','Price','PriceLimit','QuantityRelativeR','TurnoverRate',\
    'Trade','PB','PE_S','PE_D','EarningsPerShare','NetProfitDes','ROE','GrossProfitRate','NetAssetValuePerShare','CapitalStock',\
     'ScaleShareType','FinanceAnalize']
        
    parent = browser.find_element_by_xpath('//*[@id="tab"]/ul')
    boards = parent.find_elements_by_xpath('.//*')
    
    try:
        for i,board in enumerate(boards):
            board_name =board.text
            if board_name == stock_type:
                sheet = addHeaders(board_name,heads)
                board.click()
                time.sleep(3)
                print("正在获取："+board_name+"...")
                page_parent = browser.find_element_by_class_name('paginate_page')
                pages = page_parent.find_elements_by_xpath('.//*')
                total_page = int(pages[len(pages) - 1].text)

                #统计每页缺失数据的数量
                missing_count = 0
                for page in range(1,total_page+1):
                    for j in range(20):
                        printProgress("第"+str(page)+"页-"+"第"+str(j)+"条...")
                        company = browser.find_element_by_xpath('//*[@id="table_wrapper-table"]/tbody/tr['+str(j+1)+']/td[3]/a')
                        code = browser.find_element_by_xpath('//*[@id="table_wrapper-table"]/tbody/tr['+str(j+1)+']/td[2]/a')
                        price = browser.find_element_by_xpath('//*[@id="table_wrapper-table"]/tbody/tr['+str(j+1)+']/td[5]/span')
                        pl = browser.find_element_by_xpath('//*[@id="table_wrapper-table"]/tbody/tr['+str(j+1)+']/td[6]/span')
                        qrr = browser.find_element_by_xpath('//*[@id="table_wrapper-table"]/tbody/tr['+str(j+1)+']/td[15]')
                        tr = browser.find_element_by_xpath('//*[@id="table_wrapper-table"]/tbody/tr['+str(j+1)+']/td[16]')
                        
                        if price.text != '-':
                            row = (page-1)*20 + 1 + j - missing_count
                            sheet.write(row,0,code.text)
                            sheet.write(row,1,company.text)
                            sheet.write(row,2,price.text)
                            sheet.write(row,3,pl.text)
                            sheet.write(row,4,qrr.text)
                            sheet.write(row,5,tr.text)
                            getBaseData(code.text,row,6,sheet)
                        else:
                            missing_count += 1
                    page_btn = browser.find_element_by_xpath('//*[@id="main-table_paginate"]/a[2]')
                    if page < total_page:
                        page_btn.click()
                        time.sleep(3)
    except Exception as e:
        traceback.print_exc()
    browser.close()
    browser.quit()

#获取指数成分股
def getIndexStockByType(index_type):
    heads = ['StockCode','StockName','Price',\
    'Trade','PB','PE_S','PE_D','EarningsPerShare','NetProfitDes','ROE','GrossProfitRate','NetAssetValuePerShare','CapitalStock',\
     'ScaleShareType','FinanceAnalize']
    
    parent = browser.find_element_by_id("mk_type")
    
    boards = parent.find_elements_by_xpath('.//*')
    
    for i,board in enumerate(boards):
        board_name =board.text
        if board_name == index_type:
            sheet = addHeaders(board_name,heads)
            board.click()
            time.sleep(3)
            print("正在获取："+board_name+"...")
            total_page = int(browser.find_element_by_xpath('//*[@id="miniPageNav"]/b[4]/span').text)
            for page in range(1,total_page+1):
                for j in range(50):
                    printProgress("第"+str(page)+"页-"+"第"+str(j)+"条...")
                    company = browser.find_element_by_xpath('//*[@id="dt_1"]/tbody/tr['+str(j+1)+']/td[3]/a')
                    code = browser.find_element_by_xpath('//*[@id="dt_1"]/tbody/tr['+str(j+1)+']/td[2]/a')
                    price = browser.find_element_by_xpath('//*[@id="dt_1"]/tbody/tr['+str(j+1)+']/td[4]/span')
                    
                    row = (page-1)*50+1+j
                    sheet.write(row,0,code.text)
                    sheet.write(row,1,company.text)
                    sheet.write(row,2,price.text)
                    getBaseData(code.text,row,3,sheet)
                
                page_parent = browser.find_element_by_id("PageCont") 
                page_btns = page_parent.find_elements_by_xpath('.//*')
                subscript = 0
                if (len(page_btns) >0)  & (page != total_page+1):
                    for page_btn in page_btns:
                        if page_btn.text == "下一页":
                            subscript = page_btns.index(page_btn)
                    page_btns[subscript].click()
                    time.sleep(3)
                    
    browser.close()
    browser.quit()
                
def getBaseData(code,row,index,sheet):
    try:
        #设置用户代理，不然会被网站屏蔽
        headers = {"user-agent":"PostmanRuntime/7.13.0"}
        response_1 = requests.get("http://basic.10jqka.com.cn/"+code+"/company.html",headers = headers)
        response_1.encoding = 'gbk'
        e_1 = etree.HTML(response_1.text)
        # 获取行业
        try:
            trade = e_1.xpath('//*[@id="detail"]/div[2]/table/tbody/tr[2]/td[2]/span')[0]
            sheet.write(row,index,trade.text)
        except IndexError:
            try:
                trade = e_1.xpath('//*[@id="detail"]/div[3]/table/tbody/tr[2]/td[2]/span')[0]
                sheet.write(row,index,trade.text)
            except IndexError:
                sheet.write(row,index,'未知行业')
            
#         #上市时间
#         ttm = e_1.xpath('//*[@id="publish"]/div[2]/table/tbody/tr[2]/td[1]/span')[0]
#         sheet.write(row,index+1,ttm.text)

        # 获取市净率
        response_2 = requests.get("http://basic.10jqka.com.cn/"+code+"/",headers = headers)
        response_2.encoding = 'gbk'
        e_2 = etree.HTML(response_2.text)
        pb = e_2.xpath('//*[@id="sjl"]')[0]
        sheet.write(row,index+1,pb.text)
        # 获取市盈率
        # 静态市盈率
        pe_s = e_2.xpath('//*[@id="jtsyl"]')[0]
        sheet.write(row,index+2,pe_s.text)
        # 动态市盈率
        pe_d = e_2.xpath('//*[@id="dtsyl"]')[0]
        sheet.write(row,index+3,pe_d.text)
        
        #每股收益
        eps = e_2.xpath('//*[@id="profile"]/div[2]/table[2]/tbody/tr[1]/td[2]/span[2]')[0]
        sheet.write(row,index+4,eps.text)
        # 净利润
        np_des = e_2.xpath('//*[@id="profile"]/div[2]/table[2]/tbody/tr[3]/td[2]/span[2]')[0]
        sheet.write(row,index+5,np_des.text)

        # 净资产收益率
        roe = e_2.xpath('//*[@id="profile"]/div[2]/table[2]/tbody/tr[4]/td[3]/span[2]')[0]
        sheet.write(row,index+6,roe.text)
        
        #毛利率
        gpr = e_2.xpath('//*[@id="profile"]/div[2]/table[2]/tbody/tr[4]/td[2]/span[2]')[0]
        sheet.write(row,index+7,gpr.text)
        
        #每股净资产
        nvps = e_2.xpath('//*[@id="profile"]/div[2]/table[2]/tbody/tr[4]/td[1]/span[2]')[0]
        sheet.write(row,index+8,nvps.text)
        #股本
        cs = e_2.xpath('//*[@id="profile"]/div[2]/table[2]/tbody/tr[2]/td[4]/span[2]/text()')[0]
        sheet.write(row,index+9,str(cs))
        #类型
        sst = e_2.xpath('//*[@id="profile"]/div[2]/table[2]/tbody/tr[1]/td[4]/span[2]')[0]
        sheet.write(row,index+10,sst.text)
        
        #财务分析
        fa_text = ''
        fa = e_2.xpath('//*[@id="profile"]/div[2]/table[1]/tbody/tr[2]/td[2]/div[2]')
        if len(fa) > 0:
            for sub_fa in fa[0].getchildren():
                fa_text = fa_text+(sub_fa.text+',')
            sheet.write(row,index+11,fa_text)
            
    except ConnectionResetError:
        print(code)
def printProgress(content):
    sys.stdout.write("\r" +content)
    sys.stdout.flush()

In [19]:
dir_path = 'E:/wangtao/PythonWorkSpace/SpiderSpace/Stock_Companies/data/'
# dir_path = 'D:/wt/PythonWorkSpace/DataAnalysis/Stock_Companies/data/'

# getStockDataByType("沪市A股")
# workbook.save(dir_path+'上市公司沪市.xls')

# getStockDataByType("深市A股")
# workbook.save(dir_path+'上市公司深市.xls')

# getStockDataByType("中小板")
# workbook.save(dir_path+'上市公司中小板.xls')

# getStockDataByType("创业板")
# workbook.save(dir_path+'上市公司创业板.xls')

# getStockDataByType('创业板')
# workbook.save(dir_path+'创业板.xls')

# getStockDataByType('沪深A股')
# workbook.save(dir_path+'沪深A股.xls')

getStockDataByType('上证A股')
workbook.save(dir_path+'上证A股.xls')

# getStockDataByType('深证A股')
# workbook.save(dir_path+'深证A股.xls')

# getIndexStockByType('上证50')
# workbook.save(dir_path+'上证50.xls')

# getIndexStockByType('沪深300')
# workbook.save(dir_path+'沪深300.xls')

# getIndexStockByType('中证500')
# workbook.save(dir_path+'中证500.xls')

正在获取：上证A股...
第77页-第15条...

Traceback (most recent call last):
  File "<ipython-input-18-2601288fea94>", line 97, in getStockDataByType
    company = browser.find_element_by_xpath('//*[@id="table_wrapper-table"]/tbody/tr['+str(j+1)+']/td[3]/a')
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 394, in find_element_by_xpath
    return self.find_element(by=By.XPATH, value=xpath)
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 978, in find_element
    'value': value})['value']
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="table_wrapper-table"

In [42]:
headers = {"user-agent":"PostmanRuntime/7.13.0"}
# url = 'http://basic.10jqka.com.cn/002757/company.html'
url = 'http://quote.eastmoney.com/center/gridlist.html#hs_a_board'
response = requests.get(url,headers = headers)
response.encoding = response.apparent_encoding
# response.encoding = 'gbk'

# response.apparent_encoding

In [15]:
e = etree.HTML(response.text)

In [16]:
element = e.xpath('//*[@id="detail"]/div[2]/table/tbody/tr[2]/td[2]/span')

In [17]:
element[0].text

'机械设备 — 专用设备'

In [30]:
element = browser.find_element_by_xpath('//*[@id="table_wrapper-table"]/tbody/tr[1]/td[6]/span')

In [31]:
element.text

'44.00%'